In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from skimage.feature import hog, local_binary_pattern
from skimage import exposure, io, transform
import os
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Directorios de datos
testa2 = 'a2/data/test'
traina2 = 'a2/data/train'

# Obtener categorías
categorias = os.listdir(traina2)

# Parámetros HOG
pixels_per_cell_hog = (5, 5)
cells_per_block_hog = (4, 4)
orientations_hog = 13
img_size_hog = (100, 100)

# Parámetros LBP
radius_lbp = 4
n_points_lbp = 8 * radius_lbp

def calcular_hog_lbp_features(train_dir, categorias, pixels_per_cell_hog, cells_per_block_hog, orientations_hog, img_size_hog, radius_lbp, n_points_lbp):
    features_list = []
    labels = []

    for i, category in enumerate(categorias):
        category_path = os.path.join(train_dir, category)
        image_files = os.listdir(category_path)

        for image_file in image_files:
            image_path = os.path.join(category_path, image_file)

            # Cargar la imagen en escala de grises y redimensionar si es necesario
            img = io.imread(image_path, as_gray=True)
            img = transform.resize(img, img_size_hog)

            # Ajuste de contraste adaptativo
            img = exposure.equalize_adapthist(img)

            # Calcular las características HOG
            features_hog = hog(img, orientations_hog, pixels_per_cell=pixels_per_cell_hog, cells_per_block=cells_per_block_hog)

            # Calcular las características LBP
            lbp = local_binary_pattern(img, n_points_lbp, radius_lbp, method='uniform')
            features_lbp, _ = np.histogram(lbp, bins=10, range=(0, 10))

            # Concatenar características HOG y LBP
            features_combined = np.concatenate((features_hog, features_lbp))

            features_list.append(features_combined)
            labels.append(category)

    df = pd.DataFrame(features_list)
    df['label'] = labels

    return df

# Calcular características HOG y LBP
df = calcular_hog_lbp_features(traina2, categorias, pixels_per_cell_hog, cells_per_block_hog, orientations_hog, img_size_hog, radius_lbp, n_points_lbp)

# Separar características y etiquetas
X = df.drop('label', axis=1)
y = df['label']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo SVM
svm_model = SVC(kernel='rbf')  # Puedes ajustar otros parámetros aquí
svm_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo SVM con características HOG y LBP:", accuracy)


C:\Users\alero\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Precisión del modelo SVM con características HOG y LBP: 0.31666666666666665
